In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime
import os
import keras.backend as K

Using TensorFlow backend.


In [5]:
tf.reset_default_graph()

In [7]:
cur = os.getcwd()
log_dir = os.path.join(cur, 'logs', datetime.now().strftime("%d-%H-%M-%S"))
print(log_dir)

C:\Users\esther.chang\Documents\Tensorflow\logs\28-13-45-48


In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [9]:
x_train = np.reshape(x_train, [-1,28,28,1])
y_train = np.reshape(y_train, [-1,1])
x_test = np.reshape(x_test, [-1,28,28,1])
y_test = np.reshape(y_test, [-1,1])

In [13]:
def recall_func(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * K.round(y_pred), 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [14]:
model = tf.keras.Sequential()

model.add(layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.3))
model.add(layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.3))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['acc', recall_func])

tensorboard = TensorBoard(log_dir=log_dir, write_images=True, write_grads=True, histogram_freq=2)
#It won't run if set histogram fequency to nonzero
model.fit(x_train, y_train, verbose=1, epochs=10, batch_size = 64, callbacks=[tensorboard], 
         validation_data = [x_test, y_test])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
46624/60000 [======================>.......] - ETA: 58s - loss: 0.5178 - acc: 0.8137 - recall_func: 0.88

KeyboardInterrupt: 